In [7]:
import tensorflow as tf

In [8]:
from tensorflow.examples.tutorials.mnist import input_data
mnist = input_data.read_data_sets("MNIST_data/", one_hot=True)

Extracting MNIST_data/train-images-idx3-ubyte.gz
Extracting MNIST_data/train-labels-idx1-ubyte.gz
Extracting MNIST_data/t10k-images-idx3-ubyte.gz
Extracting MNIST_data/t10k-labels-idx1-ubyte.gz


In [9]:
n_input = 784  # input (28x28 pixels)
n_hidden1 = 512  # 1st hidden
n_hidden2 = 256  # 2nd hidden
n_hidden3 = 128  # 3rd hidde
n_output = 10  # output  (0-9 digits)

In [5]:
learning_rate = 1e-4
n_iterations = 1000
batch_size = 128
dropout = 0.5

In [6]:
X = tf.placeholder("float", [None, n_input])
Y = tf.placeholder("float", [None, n_output])
keep_prob = tf.placeholder(tf.float32)

In [6]:
weights = {
    'w1': tf.Variable(tf.truncated_normal([n_input, n_hidden1], stddev=0.1)),
    'w2': tf.Variable(tf.truncated_normal([n_hidden1, n_hidden2], stddev=0.1)),
    'w3': tf.Variable(tf.truncated_normal([n_hidden2, n_hidden3], stddev=0.1)),
    'out': tf.Variable(tf.truncated_normal([n_hidden3, n_output], stddev=0.1)),
}

In [7]:
biases = {
    'b1': tf.Variable(tf.constant(0.1, shape=[n_hidden1])),
    'b2': tf.Variable(tf.constant(0.1, shape=[n_hidden2])),
    'b3': tf.Variable(tf.constant(0.1, shape=[n_hidden3])),
    'out': tf.Variable(tf.constant(0.1, shape=[n_output]))
}

In [8]:
layer_1 = tf.add(tf.matmul(X, weights['w1']), biases['b1'])
layer_2 = tf.add(tf.matmul(layer_1, weights['w2']), biases['b2'])
layer_3 = tf.add(tf.matmul(layer_2, weights['w3']), biases['b3'])
layer_drop = tf.nn.dropout(layer_3, keep_prob)
output_layer = tf.matmul(layer_3, weights['out']) + biases['out']

Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.


In [9]:
cross_entropy = tf.reduce_mean(
    tf.nn.softmax_cross_entropy_with_logits(
        labels=Y, logits=output_layer
        ))
train_step = tf.train.AdamOptimizer(1e-4).minimize(cross_entropy)

Instructions for updating:

Future major versions of TensorFlow will allow gradients to flow
into the labels input on backprop by default.

See `tf.nn.softmax_cross_entropy_with_logits_v2`.



In [10]:
correct_pred = tf.equal(tf.argmax(output_layer, 1), tf.argmax(Y, 1))
accuracy = tf.reduce_mean(tf.cast(correct_pred, tf.float32))

In [11]:
init = tf.global_variables_initializer()
sess = tf.Session()
sess.run(init)

In [12]:
for i in range(n_iterations):
    batch_x, batch_y = mnist.train.next_batch(batch_size)
    sess.run(train_step, feed_dict={
        X: batch_x, Y: batch_y, keep_prob: dropout
        })

    # loss and acc per minibach
    if i % 100 == 0:
        minibatch_loss, minibatch_accuracy = sess.run(
            [cross_entropy, accuracy],
            feed_dict={X: batch_x, Y: batch_y, keep_prob: 1.0}
            )
        print(
            "Iteration ",
            str(i),
            "\t| Loss is ",
            str(minibatch_loss),
            "\t| Accuracy is ",
            str(minibatch_accuracy)
            )

Iteration 0 	| Loss = 3.2519498 	| Accuracy = 0.1171875
Iteration 100 	| Loss = 0.57492656 	| Accuracy = 0.828125
Iteration 200 	| Loss = 0.49849096 	| Accuracy = 0.875
Iteration 300 	| Loss = 0.32916522 	| Accuracy = 0.90625
Iteration 400 	| Loss = 0.29691666 	| Accuracy = 0.9296875
Iteration 500 	| Loss = 0.30811557 	| Accuracy = 0.9296875
Iteration 600 	| Loss = 0.28397188 	| Accuracy = 0.8984375
Iteration 700 	| Loss = 0.348062 	| Accuracy = 0.9296875
Iteration 800 	| Loss = 0.288534 	| Accuracy = 0.90625
Iteration 900 	| Loss = 0.27676618 	| Accuracy = 0.953125


In [13]:
test_accuracy = sess.run(accuracy, feed_dict={X: mnist.test.images, Y: mnist.test.labels, keep_prob: 1.0})
print("\nAccuracy on test set:", test_accuracy)


Accuracy on test set: 0.9172


In [14]:
from keras.models import Sequential, load_model
from keras.layers import Dense, Dropout, Flatten, Conv2D, MaxPool2D
from keras.optimizers import Adam, RMSprop
from keras.preprocessing.image import ImageDataGenerator
from keras.callbacks import ReduceLROnPlateau


def init_model():

    ###  Model Creation
    model = Sequential()
    model.add(Conv2D(filters=32, kernel_size=(5, 5), padding='Valid', activation='relu', input_shape=(28, 28, 1)))
    model.add(Conv2D(filters=32, kernel_size=(3, 3), padding='Same', activation='relu'))
    model.add(MaxPool2D(pool_size=(2, 2)))
    model.add(Dropout(0.2))

    model.add(Conv2D(filters=64, kernel_size=(5, 5), padding='Valid', activation='relu'))
    model.add(Conv2D(filters=64, kernel_size=(3, 3), padding='Same', activation='relu'))
    model.add(MaxPool2D(pool_size=(2, 2), strides=(2, 2)))
    model.add(Dropout(0.2))

    model.add(Flatten())
    model.add(Dense(519, activation="relu"))  # [[521,0.9962,70],[519,0.9969,51]
    model.add(Dropout(0.5))
    model.add(Dense(10, activation="softmax"))

    model.compile(loss='categorical_crossentropy', optimizer=Adam(lr=1e-3), metrics=["accuracy"])

    annealer = ReduceLROnPlateau(monitor='val_acc', patience=1, verbose=2, factor=0.5, min_lr=0.0000001) #patience=2

    datagen = ImageDataGenerator(
        featurewise_center=False,
        samplewise_center=False,
        featurewise_std_normalization=False,
        samplewise_std_normalization=False,
        zca_whitening=False,
        rotation_range=10,
        zoom_range=0.1,
        width_shift_range=0.1,
        height_shift_range=0.1,
        horizontal_flip=False,
        vertical_flip=False)

    return model, annealer, datagen

Using TensorFlow backend.


In [18]:
import pandas as pd
import numpy as np

import matplotlib.pyplot as plt
from sklearn.model_selection import ShuffleSplit
from sklearn.metrics import accuracy_score, confusion_matrix
from keras.utils.np_utils import to_categorical

import time
start_time = time.time()

# Get the data
train = pd.read_csv("./input/train.csv")
print(train.shape)

#Prepare data
y = train["label"]
X = train.drop("label", axis = 1)
print(y.value_counts().to_dict())
y = to_categorical(y, num_classes = 10)
del train

X = X / 255.0
X = X.values.reshape(-1,28,28,1)

# Do train and test splitting + shuffling
seed=2
train_index, valid_index = ShuffleSplit(n_splits=1,
                                        train_size=0.9,
                                        test_size=None,
                                        random_state=seed).split(X).__next__()
x_train = X[train_index]
Y_train = y[train_index]
x_test = X[valid_index]
Y_test = y[valid_index]

# Params
epochs = 30
batch_size = 64
validation_steps = 10000

# initialize Model, Annealer and Datagen
model, annealer, datagen = init_model()

# Train
train_generator = datagen.flow(x_train, Y_train, batch_size=batch_size)
test_generator = datagen.flow(x_test, Y_test, batch_size=batch_size)

history = model.fit_generator(train_generator,
                    steps_per_epoch=x_train.shape[0]//batch_size,
                    epochs=epochs,
                    validation_data=test_generator,
                    validation_steps=validation_steps//batch_size,
                    callbacks=[annealer])

score = model.evaluate(x_test, Y_test)
print('Test accuracy: ', score[1])

# Save model
model.save('Digits-1.3.0.h5')
print("Saved model to disk")

(42000, 785)
{1: 4684, 7: 4401, 3: 4351, 9: 4188, 2: 4177, 6: 4137, 0: 4132, 4: 4072, 8: 4063, 5: 3795}




Epoch 1/30
590/590 [==============================] - 180s 305ms/step - loss: 0.3545 - accuracy: 0.8856 - val_loss: 0.0330 - val_accuracy: 0.9684
Epoch 2/30


/opt/anaconda3/lib/python3.7/site-packages/keras/callbacks/callbacks.py:1042: RuntimeWarning: Reduce LR on plateau conditioned on metric `val_acc` which is not available. Available metrics are: val_loss,val_accuracy,loss,accuracy,lr
  (self.monitor, ','.join(list(logs.keys()))), RuntimeWarning


590/590 [==============================] - 179s 303ms/step - loss: 0.1036 - accuracy: 0.9677 - val_loss: 0.0473 - val_accuracy: 0.9774
Epoch 3/30
590/590 [==============================] - 180s 305ms/step - loss: 0.0797 - accuracy: 0.9765 - val_loss: 0.1076 - val_accuracy: 0.9831
Epoch 4/30
590/590 [==============================] - 188s 319ms/step - loss: 0.0659 - accuracy: 0.9804 - val_loss: 0.0138 - val_accuracy: 0.9862
Epoch 5/30
590/590 [==============================] - 186s 316ms/step - loss: 0.0599 - accuracy: 0.9817 - val_loss: 0.0758 - val_accuracy: 0.9867
Epoch 6/30
590/590 [==============================] - 187s 317ms/step - loss: 0.0564 - accuracy: 0.9827 - val_loss: 0.1313 - val_accuracy: 0.9883
Epoch 7/30
590/590 [==============================] - 188s 318ms/step - loss: 0.0485 - accuracy: 0.9852 - val_loss: 0.0060 - val_accuracy: 0.9901
Epoch 8/30
590/590 [==============================] - 182s 308ms/step - loss: 0.0463 - accuracy: 0.9857 - val_loss: 0.0258 - val_accura

KeyError: 'acc'

In [19]:
## Predict on test
test = pd.read_csv("./input/test.csv")
print(test.shape)
test = test / 255
test = test.values.reshape(-1, 28, 28, 1)
p = np.argmax(model.predict(test), axis=1)

print('Base model scores:')
valid_loss, valid_acc = model.evaluate(x_test, Y_test, verbose=0)
valid_p = np.argmax(model.predict(x_test), axis=1)
target = np.argmax(Y_test, axis=1)
cm = confusion_matrix(target, valid_p)
print(cm)

## Prepare for submission
submission = pd.DataFrame(pd.Series(range(1, p.shape[0] + 1), name='ImageId'))
submission['Label'] = p
filename="keras-cnn-{0}.csv".format(str(int(score[1]*10000)))
submission.to_csv(filename, index=False)

elapsed_time = time.time() - start_time
print("Elapsed time: {0}".format(time.strftime("%H:%M:%S", time.gmtime(elapsed_time))))

(28000, 784)
Base model scores:
[[411   0   0   0   0   0   0   0   0   0]
 [  0 483   0   0   0   0   0   1   1   0]
 [  0   0 402   0   0   0   0   1   0   0]
 [  0   1   0 412   0   2   0   0   3   0]
 [  0   0   1   0 458   0   1   0   0   1]
 [  0   0   0   0   0 371   0   0   1   0]
 [  0   0   0   0   0   1 411   0   1   0]
 [  0   0   0   0   0   0   0 446   0   0]
 [  0   0   0   0   0   0   0   0 381   1]
 [  0   0   0   0   1   0   0   1   0 407]]
Elapsed time: 01:47:29
